In [40]:
import pandas as pd
import numpy as nm
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, FloatType
from pyspark.sql.functions import col, column
from pyspark.sql.functions import expr
from pyspark.sql.functions import split
from pyspark.sql import Row
import csv
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [41]:
import os
os.environ['SPARK_LOCAL_IP'] = '192.168.1.6'

In [42]:
ss = SparkSession.builder.master("local").appName("Project").getOrCreate()

23/10/26 14:37:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [43]:
# new_file3.csv is the file generated by using Suvarna's code
data = ss.read.csv("/Users/qc/CMPSC_410_Big_Data_Project/new_file3.csv",header=True, inferSchema=False)

In [44]:
location_indexer= StringIndexer(inputCol="Location Description", outputCol="Location_Description_Index").fit(data)

#The below red bar is a Warning about the task size is large, the text disappears after finishing running

In [23]:
location_indexer

StringIndexerModel: uid=StringIndexer_d15b59d9f614, handleInvalid=error

In [24]:
transformed_data = location_indexer.transform(data)

In [25]:
transformed_data.show(4)

+--------+-----------+--------------------+-------------------+----+------------+--------------------+--------------------+------+--------+--------+----+--------------+--------+------+------------+-------------+--------------------------+
|      ID|Case Number|                Date|              Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|District|Ward|Community Area|FBI Code|  Year|    Latitude|    Longitude|Location_Description_Index|
+--------+-----------+--------------------+-------------------+----+------------+--------------------+--------------------+------+--------+--------+----+--------------+--------+------+------------+-------------+--------------------------+
|12045583|   JD226426|05/07/2020 10:24:...|035XX S INDIANA AVE|0820|       THEFT|      $500 AND UNDER|           APARTMENT| false|   false|     2.0| 3.0|          35.0|      06|2020.0|41.830481843|-87.621751752|                       1.0|
|12031001|   JD209965|04/16/2020 05:00:...| 

In [26]:
type_indexer = StringIndexer(inputCol="Primary Type", outputCol="Primary_Type_Index").fit(data)

In [27]:
type_indexer

StringIndexerModel: uid=StringIndexer_d238ded589d8, handleInvalid=error

In [28]:
transformed_data = type_indexer.transform(data)

In [29]:
transformed_data.show(4)

+--------+-----------+--------------------+-------------------+----+------------+--------------------+--------------------+------+--------+--------+----+--------------+--------+------+------------+-------------+------------------+
|      ID|Case Number|                Date|              Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|District|Ward|Community Area|FBI Code|  Year|    Latitude|    Longitude|Primary_Type_Index|
+--------+-----------+--------------------+-------------------+----+------------+--------------------+--------------------+------+--------+--------+----+--------------+--------+------+------------+-------------+------------------+
|12045583|   JD226426|05/07/2020 10:24:...|035XX S INDIANA AVE|0820|       THEFT|      $500 AND UNDER|           APARTMENT| false|   false|     2.0| 3.0|          35.0|      06|2020.0|41.830481843|-87.621751752|               0.0|
|12031001|   JD209965|04/16/2020 05:00:...|    005XX W 32ND ST|0460|     BAT

In [30]:
input_features = ['District', 'Ward', 'Community Area', 'Year', 'Latitude', 'Longitude']

In [31]:
assembler = VectorAssembler(inputCols=input_features, outputCol="features")

In [32]:
assembler

VectorAssembler_d34e96a69816

In [33]:
vectorized_data = assembler.transform(transformed_data)

In [37]:
vectorized2_data = vectorized_data.select("features", '`Primary Type`', '`Location Description`')
vectorized2_data.show(5)

+--------------------+------------+--------------------+
|            features|Primary Type|Location Description|
+--------------------+------------+--------------------+
|[2.0,3.0,35.0,202...|       THEFT|           APARTMENT|
|[9.0,11.0,60.0,20...|     BATTERY|           APARTMENT|
|[4.0,7.0,46.0,202...|     ASSAULT|              STREET|
|[7.0,15.0,67.0,20...|     BATTERY|RESIDENCE - PORCH...|
|[7.0,16.0,68.0,20...|     BATTERY|              STREET|
+--------------------+------------+--------------------+
only showing top 5 rows

